# Note

**Since only one molecular function was selected by feature selection we decided to just outright ignore it and build models with just the selected drug descriptors and protein sequence descriptors.**

# Imports

In [27]:
# General Imports
import pandas as pd
import numpy as np
from modify_dataset import load_from_pickle, load_to_pickle

# Plotting libraries
import plotly.express as px

# Dimensionality reduction
from sklearn.decomposition import PCA

# Model selection
from sklearn.model_selection import permutation_test_score

# Scalers
from sklearn.preprocessing import StandardScaler

# Metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, matthews_corrcoef
from sklearn.metrics import mean_absolute_error, r2_score

# Classification Models
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Regression Models
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

# Error Analysis
# import eli5
# from lime.lime_tabular import LimeTabularExplainer

# Scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Other Model Stuff
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# Training & Test Sets

In [28]:
feature_selection_columns = np.load("Dataset_Files/Feature_Selection/features_dd_psd_list.npy",
                                    allow_pickle=True)

In [29]:
X_train = np.load("Dataset_Files/Training_Test_Sets/X_train_dd_psd_feature_selection.npy")
y_train = np.load("Dataset_Files/Training_Test_Sets/y_train_dd_psd.npy")

X_test_classification = np.load("Dataset_Files/Training_Test_Sets/X_test_classification_dd_psd_feature_selection.npy")
y_test_classification = np.load("Dataset_Files/Training_Test_Sets/y_test_classification.npy")

In [30]:
# Useful Information & Sanity Checks
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape[0]}")

print(f"X_test_classification shape: {X_test_classification.shape}")
print(f"y_test_classification shape: {y_test_classification.shape[0]} ", end="")
print(f"(Binding Count: {y_test_classification[y_test_classification == 1].shape[0]}, ", end="")
print(f"Non-Binding Count: {y_test_classification[y_test_classification == 0].shape[0]})")

X_train shape: (134734, 1044)
y_train shape: 134734
X_test_classification shape: (30141, 1044)
y_test_classification shape: 30141 (Binding Count: 22001, Non-Binding Count: 8140)


# Helper Functions

In [31]:
# Called when gridsearch is not used
def calculate_metrics_classification(y_true, y_pred):
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    print(f"F1 score: {f1}")
    print(f"Matthews Correlation Coefficient: {mcc}")
    print(f"Accuracy score: {accuracy}")
    print(f"Recall score: {recall}")
    print(f"Precision score: {precision}")


# Called when gridsearch is used
def best_model_information_classification(model):
    print(f"Best model index: {model.best_index_}")
    print(f"Best parameters: {model.best_params_}")
    print(f"Best F1 score: {model.best_score_}")
    print(f"Matthews Correlation Coefficient: {model.cv_results_['mean_test_mcc'][model.best_index_]}")
    print(f"Accuracy score: {model.cv_results_['mean_test_accuracy'][model.best_index_]}")
    print(f"Recall score: {model.cv_results_['mean_test_recall'][model.best_index_]}")
    print(f"Precision score: {model.cv_results_['mean_test_precision'][model.best_index_]}")


# The common p-value of 0.0099009900990099011 is returned when no model with the shuffled labels produces a better score
# https://github.com/scikit-learn/scikit-learn/issues/8379
def permutation_test_classification(model, repetitions=100):
    score, perm_scores, pvalue = permutation_test_score(model,
                                                        X_train,
                                                        y_train,
                                                        scoring=make_scorer(matthews_corrcoef),
                                                        cv=10,
                                                        n_permutations=repetitions,
                                                        n_jobs=-1,
                                                        random_state=42,
                                                        )

    permutation_test_dataframe = pd.DataFrame(data={"score": score,
                                                    "perm_scores": perm_scores,
                                                    "pvalue": pvalue}
                                              )

    fig = px.histogram(permutation_test_dataframe,
                       x="perm_scores",
                       histnorm='probability density',
                       labels={'probability density': 'Probability Density',
                               'perm_scores': 'Perm Scores'}
                       )
    fig.update_xaxes(range=[0, 1], dtick=0.1)
    fig.add_vline(x=score,
                  line_dash='dash',
                  line_color='firebrick',
                  annotation_text=f"Original Data MCC: {round(score, 4)} <br> (P-Value: {round(pvalue, 6)})",
                  annotation_position="top right",
                  annotation_font_size=14)
    fig.show()


def prediction_category_classification(df):
    if df['True Class'] == df['Prediction']:
        return 'Correct'
    elif (df['True Class'] == 0) and (df['Prediction'] == 1):
        return 'False Positive'
    else:
        return 'False Negative'


def error_analysis_classification(y_pred):
    X_set = pd.DataFrame(X_test_classification, columns=feature_selection_columns)
    y_set = pd.Series(y_test_classification, name="Activity_Binary")

    # Combining data into one dataframe
    y_pred_series = pd.Series(y_pred, index=y_set.index)

    error_analysis_dataframe = pd.concat([X_set, y_set], axis=1)
    error_analysis_dataframe = pd.concat([error_analysis_dataframe, y_pred_series], axis=1)
    error_analysis_dataframe.rename(columns={"Activity_Binary": "True Class", 0: "Prediction"}, inplace=True)
    error_analysis_dataframe["Is the prediction correct?"] = error_analysis_dataframe.apply(
        prediction_category_classification, axis=1)

    # Scaling
    scaler = StandardScaler()
    scaler.fit(error_analysis_dataframe.loc[:, feature_selection_columns])
    scaled_data = scaler.transform(error_analysis_dataframe.loc[:, feature_selection_columns])

    # PCA
    pca = PCA(n_components=2, random_state=0)
    pca.fit(scaled_data)
    pca_data = pca.transform(scaled_data)
    pca_dataframe_2d = pd.DataFrame(pca_data, columns=["PCA_Dimension_1", "PCA_Dimension_2"], index=y_set.index)

    # Joining dataframes
    error_analysis_dataframe = pd.concat([error_analysis_dataframe, pca_dataframe_2d], axis=1)

    # Plot
    fig = px.scatter(error_analysis_dataframe, x="PCA_Dimension_1", y="PCA_Dimension_2",
                     color="Is the prediction correct?",
                     symbol="Is the prediction correct?",
                     hover_data=['MW', 'TPSA', 'XLogP', 'NHD', 'NHA', 'NRB', 'True Class', 'Prediction'],
                     title="Correct Classifications vs Misclassifications")
    fig.show()

    # Useful stats
    print(
        f"Number of correct classifications: {len(error_analysis_dataframe[error_analysis_dataframe['Is the prediction correct?'] == 'Correct'])}")
    print(
        f"Number of misclassifications: {len(error_analysis_dataframe[error_analysis_dataframe['Is the prediction correct?'] != 'Correct'])}")
    print(
        f"False Positives (True class:0, Prediction:1): {len(error_analysis_dataframe[(error_analysis_dataframe['True Class'] == 0) & (error_analysis_dataframe['Prediction'] == 1)])}")
    print(
        f"False Negatives (True class:1, Prediction:0): {len(error_analysis_dataframe[(error_analysis_dataframe['True Class'] == 1) & (error_analysis_dataframe['Prediction'] == 0)])}")

    return error_analysis_dataframe.sort_values('Is the prediction correct?')


# def model_weights_classification(model, classification_group):
#     if classification_group == 'cd':
#         return eli5.show_weights(model,
#                                  feature_names=X_train_cd.columns,
#                                  target_names={1:"BBB+",0:"BBB-"})
#     elif classification_group == 'cd_se_i':
#         return eli5.show_weights(model,
#                                  feature_names=X_train_cd_se_i.loc[:,feature_selection_support].columns,
#                                  target_names={1:"BBB+",0:"BBB-"})
#     else:
#         raise ValueError("Invalid group. Please choose 'cd' or 'cd_se_i'")

# def get_lime_explainer_classification(classification_group):
#     if classification_group == 'cd':
#         X_train = X_train_cd
#         y_train = y_train_cd
#     elif classification_group == 'cd_se_i':
#         X_train = X_train_cd_se_i.loc[:,feature_selection_support]
#         y_train = y_train_cd_se_i
#     else:
#         raise ValueError("Invalid group. Please choose 'cd' or 'cd_se_i'")
#
#     explainer = LimeTabularExplainer(training_data=np.array(X_train),
#                                      mode='classification',
#                                      feature_names=list(X_train.columns),
#                                      training_labels=y_train,
#                                      class_names=['BBB-','BBB+'],
#                                      random_state=42)
#     return explainer

grid_search_scoring_dict = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'accuracy': make_scorer(accuracy_score),
    'mcc': make_scorer(matthews_corrcoef),
}

# Model Training & Testing

In [32]:
def on_step(optim_result):
    global index
    print(f"Iteration Completed: {index}:{model.total_iterations}")
    index += 1

## Dummy Classifier (DC)

In [33]:
dummy_classifier = Pipeline(
    [
        ('scale', StandardScaler()),
        ('model', DummyClassifier(random_state=42))
    ]
)
dummy_classifier.get_params()

{'memory': None,
 'steps': [('scale', StandardScaler()),
  ('model', DummyClassifier(random_state=42))],
 'verbose': False,
 'scale': StandardScaler(),
 'model': DummyClassifier(random_state=42),
 'scale__copy': True,
 'scale__with_mean': True,
 'scale__with_std': True,
 'model__constant': None,
 'model__random_state': 42,
 'model__strategy': 'prior'}

### Training

In [34]:
dummy_classifier.fit(X_train, y_train)

y_train_pred = dummy_classifier.predict(X_train)
calculate_metrics_classification(y_train, y_train_pred)

# Save Model
dump(dummy_classifier, 'Dataset_Files/Classification_Models/dc.joblib')

F1 score: 0.8459466288646107
Matthews Correlation Coefficient: 0.0
Accuracy score: 0.7330221028099811
Recall score: 1.0
Precision score: 0.7330221028099811


['Dataset_Files/Classification_Models/dc.joblib']

### Testing

In [35]:
# Load Model
dummy_classifier = load('Dataset_Files/Classification_Models/dc.joblib')

y_test_pred = dummy_classifier.predict(X_test_classification)
calculate_metrics_classification(y_test_classification, y_test_pred)

F1 score: 0.8438878447316942
Matthews Correlation Coefficient: 0.0
Accuracy score: 0.729935967618858
Recall score: 1.0
Precision score: 0.729935967618858


## Logistic Regression (LR)

In [11]:
pipe = Pipeline(
    [
        ('scale', StandardScaler()),
        ('model', LogisticRegression(random_state=42))
    ]
)
pipe.get_params()

{'memory': None,
 'steps': [('scale', StandardScaler()),
  ('model', LogisticRegression(random_state=42))],
 'verbose': False,
 'scale': StandardScaler(),
 'model': LogisticRegression(random_state=42),
 'scale__copy': True,
 'scale__with_mean': True,
 'scale__with_std': True,
 'model__C': 1.0,
 'model__class_weight': None,
 'model__dual': False,
 'model__fit_intercept': True,
 'model__intercept_scaling': 1,
 'model__l1_ratio': None,
 'model__max_iter': 100,
 'model__multi_class': 'auto',
 'model__n_jobs': None,
 'model__penalty': 'l2',
 'model__random_state': 42,
 'model__solver': 'lbfgs',
 'model__tol': 0.0001,
 'model__verbose': 0,
 'model__warm_start': False}

In [12]:
model = BayesSearchCV(estimator=pipe,
                      search_spaces=[
                          {'model__C': Real(1e-6, 1e+2, prior='log-uniform'),
                           'model__solver': Categorical(['newton-cg','lbfgs','sag']),
                           'model__penalty': Categorical(['none', 'l2']),
                           'model__max_iter': Integer(50, 5000),
                           'model__class_weight': Categorical([None, "balanced"])},
                          {'model__C': Real(1e-6, 1e+2, prior='log-uniform'),
                           'model__solver': Categorical(['liblinear']),
                           'model__penalty': Categorical(['l2', 'l1']),
                           'model__max_iter': Integer(50, 5000),
                           'model__class_weight': Categorical([None, "balanced"])},
                          {'model__C': Real(1e-6, 1e+2, prior='log-uniform'),
                           'model__l1_ratio': Real(0,1),
                           'model__solver': Categorical(['saga']),
                           'model__penalty': Categorical(['none', 'l2', 'l1', 'elasticnet']),
                           'model__max_iter': Integer(50, 5000),
                           'model__class_weight': Categorical([None, "balanced"])},
                      ],
                      scoring='f1',
                      cv=5,
                      error_score=np.nan,
                      n_jobs=-1,
                      pre_dispatch='2*n_jobs',
                      random_state=42)

### Training

In [ ]:
index = 1
model.fit(X_train, y_train)

optimised_lr = model.best_estimator_
best_model_information_classification(model)

# Save Model & CV Results
dump(optimised_lr, 'Classification_Models/optimised_lr.joblib')
np.save("Dataset_Files/Classification_Models/optimised_lr_cv_results", model.cv_results_)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


In [ ]:
logistic_regression_grid_search_dataframe = pd.DataFrame(
    np.load("Dataset_Files/Classification_Models/optimised_lr_cv_results").tolist())
logistic_regression_grid_search_dataframe.sort_values(by=["rank_test_f1"], inplace=True)
logistic_regression_grid_search_dataframe

### Permutation Testing

In [ ]:
# Load Model
optimised_lr = load('Dataset_Files/Classification_Models/optimised_lr.joblib')
permutation_test_classification(optimised_lr)

### Testing

In [ ]:
# Load Model
optimised_lr = load('Dataset_Files/Classification_Models/optimised_lr.joblib')

y_test_pred = optimised_lr.predict(X_test_classification)
calculate_metrics_classification(y_test_classification, y_test_pred)

## Support Vector Classification (SVC)

In [ ]:
pipe = Pipeline(
    [
        ('scale', StandardScaler()),
        ('model', SVC(random_state=42))
    ]
)
pipe.get_params()

In [ ]:
model = BayesSearchCV(estimator=pipe,
                      search_spaces=
                      {'model__C': Real(1e-6, 1e+2, prior='log-uniform'),
                       'model__kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']),
                       'model__degree': Integer(2, 8),
                       'model__gamma': Categorical(['scale', 'auto']),
                       'model__class_weight': Categorical([None, "balanced"])},
                      scoring='f1',
                      cv=5,
                      error_score=np.nan,
                      n_jobs=-1,
                      pre_dispatch='2*n_jobs',
                      random_state=42)

### Training

In [ ]:
index = 1
model.fit(X_train, y_train)

optimised_svc = model.best_estimator_
best_model_information_classification(model)

# Save Model & CV Results
dump(optimised_svc, 'Classification_Models/optimised_svc.joblib')
np.save("Dataset_Files/Classification_Models/optimised_svc_cv_results", model.cv_results_)

In [ ]:
svc_grid_search_dataframe = pd.DataFrame(
    np.load("Dataset_Files/Classification_Models/optimised_svc_cv_results").tolist())
svc_grid_search_dataframe.sort_values(by=["rank_test_f1"], inplace=True)
svc_grid_search_dataframe

### Permutation Testing

In [ ]:
# Load Model
optimised_svc = load('Dataset_Files/Classification_Models/optimised_svc.joblib')
permutation_test_classification(optimised_svc)

### Testing

In [ ]:
# Load Model
optimised_svc = load('Dataset_Files/Classification_Models/optimised_svc.joblib')

y_test_pred = optimised_svc.predict(X_test_classification)
calculate_metrics_classification(y_test_classification, y_test_pred)

## K-Nearest Neighbors Classifier (KNNC)

In [ ]:
pipe = Pipeline(
    [
        ('scale', StandardScaler()),
        ('model', KNeighborsClassifier())
    ]
)
pipe.get_params()

In [ ]:
model = BayesSearchCV(estimator=pipe,
                      search_spaces=
                      {'model__n_neighbors': Integer(4, 20),
                       'model__weights': Categorical(['uniform', 'distance']),
                       'model__algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
                       },
                      scoring='f1',
                      cv=5,
                      error_score=np.nan,
                      n_jobs=-1,
                      pre_dispatch='2*n_jobs',
                      random_state=42)

### Training

In [ ]:
index = 1
model.fit(X_train, y_train)

optimised_knnc = model.best_estimator_
best_model_information_classification(model)

# Save Model & CV Results
dump(optimised_knnc, 'Classification_Models/optimised_knnc.joblib')
np.save("Dataset_Files/Classification_Models/optimised_knnc_cv_results", model.cv_results_)

In [ ]:
knnc_grid_search_dataframe = pd.DataFrame(
    np.load("Dataset_Files/Classification_Models/optimised_knnc_cv_results").tolist())
knnc_grid_search_dataframe.sort_values(by=["rank_test_f1"], inplace=True)
knnc_grid_search_dataframe

### Permutation Testing

In [ ]:
# Load Model
optimised_knnc = load('Dataset_Files/Classification_Models/optimised_knnc.joblib')
permutation_test_classification(optimised_knnc)

### Testing

In [ ]:
# Load Model
optimised_knnc = load('Dataset_Files/Classification_Models/optimised_knnc.joblib')

y_test_pred = optimised_knnc.predict(X_test_classification)
calculate_metrics_classification(y_test_classification, y_test_pred)

## Decision Tree Classifier (DTC)

In [ ]:
pipe = Pipeline(
    [
        ('scale', StandardScaler()),
        ('model', DecisionTreeClassifier(random_state=42))
    ]
)
pipe.get_params()

In [ ]:
model = BayesSearchCV(estimator=pipe,
                      search_spaces=
                      {'model__criterion': Categorical(['gini', 'entropy']),
                       'model__splitter': Categorical(['best', 'random']),
                       'model__max_features': Categorical([None, 'sqrt', 'log2']),
                       'model__class_weight': Categorical([None, 'balanced'])
                       },
                      scoring='f1',
                      cv=5,
                      error_score=np.nan,
                      n_jobs=-1,
                      pre_dispatch='2*n_jobs',
                      random_state=42)

### Training

In [ ]:
index = 1
model.fit(X_train, y_train)

optimised_dtc = model.best_estimator_
best_model_information_classification(model)

# Save Model & CV Results
dump(optimised_dtc, 'Classification_Models/optimised_dtc.joblib')
np.save("Dataset_Files/Classification_Models/optimised_dtc_cv_results", model.cv_results_)

In [ ]:
dtc_grid_search_dataframe = pd.DataFrame(
    np.load("Dataset_Files/Classification_Models/optimised_dtc_cv_results").tolist())
dtc_grid_search_dataframe.sort_values(by=["rank_test_f1"], inplace=True)
dtc_grid_search_dataframe

### Permutation Testing

In [ ]:
# Load Model
optimised_dtc = load('Dataset_Files/Classification_Models/optimised_dtc.joblib')
permutation_test_classification(optimised_dtc)

### Testing

In [ ]:
# Load Model
optimised_dtc = load('Dataset_Files/Classification_Models/optimised_dtc.joblib')

y_test_pred = optimised_dtc.predict(X_test_classification)
calculate_metrics_classification(y_test_classification, y_test_pred)

## Random Forest Classifier (RFC)

In [5]:
pipe = Pipeline(
    [
        ('scale', StandardScaler()),
        ('model', RandomForestClassifier(random_state=42))
    ]
)
pipe.get_params()

{'memory': None,
 'steps': [('scale', StandardScaler()),
  ('model', RandomForestClassifier(random_state=42))],
 'verbose': False,
 'scale': StandardScaler(),
 'model': RandomForestClassifier(random_state=42),
 'scale__copy': True,
 'scale__with_mean': True,
 'scale__with_std': True,
 'model__bootstrap': True,
 'model__ccp_alpha': 0.0,
 'model__class_weight': None,
 'model__criterion': 'gini',
 'model__max_depth': None,
 'model__max_features': 'sqrt',
 'model__max_leaf_nodes': None,
 'model__max_samples': None,
 'model__min_impurity_decrease': 0.0,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__min_weight_fraction_leaf': 0.0,
 'model__n_estimators': 100,
 'model__n_jobs': None,
 'model__oob_score': False,
 'model__random_state': 42,
 'model__verbose': 0,
 'model__warm_start': False}

In [6]:
model = BayesSearchCV(estimator=pipe,
                      search_spaces=
                      {'model__n_estimators': Integer(100, 800),
                       'model__criterion': Categorical(['gini', 'entropy', 'log_loss']),
                       'model__max_features': Categorical([None, 'sqrt', 'log2']),
                       'model__class_weight': Categorical([None, 'balanced', 'balanced_subsample'])
                       },
                      scoring='f1',
                      cv=5,
                      error_score=np.nan,
                      n_jobs=-1,
                      pre_dispatch='2*n_jobs',
                      random_state=42)

### Training

In [ ]:
index = 1
model.fit(X_train, y_train)

optimised_rfc = model.best_estimator_
best_model_information_classification(model)

# Save Model
dump(optimised_rfc, 'Classification_Models/optimised_rfc.joblib')
np.save("Dataset_Files/Classification_Models/optimised_rfc_cv_results", model.cv_results_)

Fitting 10 folds for each of 135 candidates, totalling 1350 fits


In [ ]:
rfc_grid_search_dataframe = pd.DataFrame(
    np.load("Dataset_Files/Classification_Models/optimised_rfc_cv_results").tolist())
rfc_grid_search_dataframe.sort_values(by=["rank_test_f1"], inplace=True)
rfc_grid_search_dataframe

### Permutation Testing

In [ ]:
# Load Model
optimised_rfc = load('Dataset_Files/Classification_Models/optimised_rfc.joblib')
permutation_test_classification(optimised_rfc)

### Testing

In [ ]:
# Load Model
optimised_rfc = load('Dataset_Files/Classification_Models/optimised_rfc.joblib')

y_test_pred = optimised_rfc.predict(X_test_classification)
calculate_metrics_classification(y_test_classification, y_test_pred)

## Stochastic Gradient Descent Classifier (SGDC)

In [8]:
pipe = Pipeline(
    [
        ('scale', StandardScaler()),
        ('model', SGDClassifier(random_state=42))
    ]
)
pipe.get_params()

{'memory': None,
 'steps': [('scale', StandardScaler()),
  ('model', SGDClassifier(random_state=42))],
 'verbose': False,
 'scale': StandardScaler(),
 'model': SGDClassifier(random_state=42),
 'scale__copy': True,
 'scale__with_mean': True,
 'scale__with_std': True,
 'model__alpha': 0.0001,
 'model__average': False,
 'model__class_weight': None,
 'model__early_stopping': False,
 'model__epsilon': 0.1,
 'model__eta0': 0.0,
 'model__fit_intercept': True,
 'model__l1_ratio': 0.15,
 'model__learning_rate': 'optimal',
 'model__loss': 'hinge',
 'model__max_iter': 1000,
 'model__n_iter_no_change': 5,
 'model__n_jobs': None,
 'model__penalty': 'l2',
 'model__power_t': 0.5,
 'model__random_state': 42,
 'model__shuffle': True,
 'model__tol': 0.001,
 'model__validation_fraction': 0.1,
 'model__verbose': 0,
 'model__warm_start': False}

In [ ]:
model = BayesSearchCV(estimator=pipe,
                      search_spaces=
                      {'model__loss': Categorical(
                          ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error',
                           'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']),
                       'model__penalty': Categorical(['l2', 'l1', 'elasticnet']),
                       'model__alpha': Real(1e-6, 1e-1, prior='log-uniform'),
                       'model__learning_rate': Categorical(['constant', 'optimal', 'invscaling', 'adaptive']),
                       'model__class_weight': Categorical([None, 'balanced'])
                       },
                      scoring='f1',
                      cv=5,
                      error_score=np.nan,
                      n_jobs=-1,
                      pre_dispatch='2*n_jobs',
                      random_state=42)

### Training

In [ ]:
index = 1
model.fit(X_train, y_train)

optimised_sgdc = model.best_estimator_
best_model_information_classification(model)

# Save Model
dump(optimised_sgdc, 'Classification_Models/optimised_sgdc.joblib')
np.save("Dataset_Files/Classification_Models/optimised_sgdc_cv_results", model.cv_results_)

In [ ]:
sgdc_grid_search_dataframe = pd.DataFrame(
    np.load("Dataset_Files/Classification_Models/optimised_sgdc_cv_results").tolist())
sgdc_grid_search_dataframe.sort_values(by=["rank_test_f1"], inplace=True)
sgdc_grid_search_dataframe

### Permutation Testing

In [ ]:
# Load Model
optimised_sgdc = load('Dataset_Files/Classification_Models/optimised_sgdc.joblib')
permutation_test_classification(optimised_sgdc)

### Testing

In [ ]:
# Load Model
optimised_sgdc = load('Dataset_Files/Classification_Models/optimised_sgdc.joblib')

y_test_pred = optimised_sgdc.predict(X_test_classification)
calculate_metrics_classification(y_test_classification, y_test_pred)